In [3]:
import tensorflow as tf
from tensorflow import keras
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [4]:
model = keras.models.load_model('model_epoch_17_val_acc_0.9231.keras')

I0000 00:00:1730286418.825160   53594 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730286419.008295   53594 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730286419.008410   53594 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730286419.014263   53594 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730286419.014337   53594 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [5]:
from keras.preprocessing.image import load_img
from keras.applications.resnet import preprocess_input
import numpy as np

In [6]:
img = load_img('./sp1-P1.jpg', target_size=(224, 224))

In [7]:
x = np.array(img)
X = np.array([x])
X.shape

(1, 224, 224, 3)

In [8]:
X = preprocess_input(X)
pred = model.predict(X)
pred

I0000 00:00:1730286423.071607   56806 service.cc:146] XLA service 0x7f6364002910 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730286423.071646   56806 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2024-10-30 18:07:03.156968: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-30 18:07:03.545918: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90101


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


I0000 00:00:1730286425.965626   56806 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


array([[0.09367929, 0.9063207 ]], dtype=float32)

In [9]:
pred = {"prediction": pred[0][1]}
pred

{'prediction': 0.9063207}

# Convert Keras to TF-Lite

In [10]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('model_hw_newhandpd_aug-ilum_rasnet50.tflite', 'wb') as f_out:
    f_out.write(tflite_model)


Saved artifact at '/tmp/tmpbzfn23id'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_3')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  140069008949712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140069008951248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140069008950288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140069008949904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140069008951056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140069008950672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140069008951824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140069008952976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140069008952400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140069008953168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1400690089533

W0000 00:00:1730286432.734329   53594 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1730286432.734397   53594 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-10-30 18:07:12.735022: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpbzfn23id
2024-10-30 18:07:12.741831: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-10-30 18:07:12.741868: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpbzfn23id
2024-10-30 18:07:12.822919: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-10-30 18:07:12.840417: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-10-30 18:07:13.591032: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpbzfn23id
2024-10-30 18:07:13.742972: I tensorflow/cc/saved_model/loader.cc

In [11]:
import tensorflow.lite as tflite

In [12]:
interpreter = tflite.Interpreter(model_path='model_hw_newhandpd_aug-ilum_rasnet50.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [13]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [15]:
pred = {"prediction": preds[0][1]}
pred

{'prediction': 0.9063211}

# Removing TF dependency

In [16]:
from PIL import Image

In [17]:
with Image.open('./sp1-P1.jpg') as img:
    img = img.resize((224, 224), Image.NEAREST)

In [20]:
def preprocess_input(x):
    x = x[..., ::-1]
    mean = [103.939, 116.779, 123.68]
    x[..., 0] -= mean[0]
    x[..., 1] -= mean[1]
    x[..., 2] -= mean[2]
    return x

In [21]:
x = np.array(img, dtype='float32')
X = np.array([x])

X = preprocess_input(X)

In [22]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
pred = {"prediction": preds[0][1]}
pred

{'prediction': 0.9063211}

In [23]:
!pip install keras-image-helper

In [24]:
!pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

Looking in indexes: https://pypi.org/simple, https://google-coral.github.io/py-repo/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 423.9 kB/s eta 0:00:0000:0100:01


In [1]:
import tflite_runtime.interpreter as tflite
from keras_image_helper import create_preprocessor

In [2]:
interpreter = tflite.Interpreter(model_path='model_hw_newhandpd_aug-ilum_rasnet50.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
preprocessor = create_preprocessor('resnet50', target_size=(224, 224))

In [4]:
path = './sp1-P1.jpg'
X = preprocessor.from_path(path)

In [5]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
pred = {"prediction": preds[0][1]}
pred

{'prediction': 0.90632087}